In [23]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [24]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [25]:
def scrape_links(page_number):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://jatimnow.com/category/politik/{page_number}"
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h2',{"class": "entry-title"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [26]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 8 links from page 1
Total Links: 8


In [27]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "entry-title"})
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # Author berita
            author_elem =soup.find('span', {"class": "cat-links-content"})  
            if author_elem:
                author_text = author_elem.text.split(':')[2]
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('time', {"itemprop": "datePublished"})
            if date_elem:
                date_text = date_elem.text
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('span', {"class": "kanal"})
            # if category_elements:
            #     category_text= category_elements.text
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "entry-content entry-content-single clearfix"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'jatim',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [28]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [29]:
df = pd.DataFrame(results)
print(len(results))
df.head(5)

8


,title,author,category,date,content,region,link
0,Ribuan Relawan Adies Kadir Lantunkan Selawat u...,Ni'am Kurniawan,Category not found,"Minggu, 24 Sep 2023 09:25 WIB",jatimnow.com - Lantunan selawat menggema dari ...,jatim,https://jatimnow.com/baca-61867-ribuan-relawan...
1,5 Partai Parlemener Usung Petahana di Dapil 2 ...,Ahaddiini HM,Category not found,"Jumat, 22 Sep 2023 14:53 WIB",jatimnow.com - Dari 18 partai dengan daftar ca...,jatim,https://jatimnow.com/baca-61822-5-partai-parle...
2,Keluarga Tambak Beras Doa Bersama untuk Yenny ...,Ni'am Kurniawan,Category not found,"Minggu, 24 Sep 2023 07:03 WIB",jatimnow.com - Puluhan gus keluarga besar Ponp...,jatim,https://jatimnow.com/baca-61860-keluarga-tamba...
3,"Khofifah, Emil dan Fauzi Disebut Memiliki Kans...",Ni'am Kurniawan,Category not found,"Jumat, 22 Sep 2023 20:55 WIB",jatimnow.com - Direktur Lembaga riset ARCI Bai...,jatim,https://jatimnow.com/baca-61838-khofifah-emil-...
4,Relawan Ganjar Pasang Lampu PJU di Bojonegoro,Misbahul Munir,Category not found,"Minggu, 24 Sep 2023 16:19 WIB",jatimnow.com - Relawan Orang Muda Ganjar (OMG)...,jatim,https://jatimnow.com/baca-61881-relawan-ganjar...


In [ ]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../../tempat_hasil_daerah/jatim_now_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')